In [1]:
# Imports
import os
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [4]:
# Join all 7 dataframes along mutually shared columns
cwd = os.getcwd()
dfs = []
for filename in os.listdir(cwd):
    if filename.endswith('.csv') and filename != 'data.csv':
        filepath = os.path.join(cwd, filename)
        df = pd.read_csv(filepath)
        dfs.append(df)
shared_columns = set(dfs[0].columns)
for df in dfs:
    shared_columns = shared_columns.intersection(df.columns)
df = pd.concat([df[list(shared_columns)] for df in dfs], ignore_index=True)

# Reset index & display results
df = df.reset_index(drop=True)
df.head()

,[Students]Custom Field1,[CenterReasons]Reason,[Visits]Time In,[Sections]Term ID,[Students]Class,[Visits]Time Out,[Subcenters]Name,[Visits]Date In,[Students]College,[Students]Major,[Students]Ethnicity,[Students]EMCCGPA,[Students]Custom Field6,[Faculty]FullName
0,AF :: FOP Freshman-no prev coll cred,Writing,16:42:29,201830,JR,18:16:05,Reed Fourth,11/13/2018,Liberal Arts&Sciences,English,White,2.12,Baseball,"Ploetz, Elmer"
1,AF :: FOP Freshman-no prev coll cred,Writing,11:55:05,201830,JR,13:25:05,Reed Fourth,11/26/2018,Liberal Arts&Sciences,English,White,2.12,Baseball,"FitzPatrick, Cornelius"
2,AF :: FOP Freshman-no prev coll cred,Writing,11:59:45,201830,JR,14:56:30,Reed Fourth,12/3/2018,Liberal Arts&Sciences,English,White,2.12,Baseball,"FitzPatrick, Cornelius"
3,AF :: FOP Freshman-no prev coll cred,Writing,13:58:04,201830,JR,15:17:00,Reed Fourth,12/4/2018,Liberal Arts&Sciences,English,White,2.12,Baseball,"FitzPatrick, Cornelius"
4,AF :: FOP Freshman-no prev coll cred,Writing,13:34:57,201830,JR,15:04:57,Reed Fourth,12/5/2018,Liberal Arts&Sciences,English,White,2.12,Baseball,"FitzPatrick, Cornelius"


In [5]:
# Convert datetime columns & throw out errors
try:
    df['[Visits]Time In'] = pd.to_datetime(df['[Visits]Time In'], format='%H:%M:%S')
except ValueError:
    problematic_rows = pd.to_datetime(df['[Visits]Time In'], format='%H:%M:%S', errors='coerce').isna()
    df = df.loc[~problematic_rows]
    df['[Visits]Time In'] = pd.to_datetime(df['[Visits]Time In'], format='%H:%M:%S')
try:
    df['[Visits]Time Out'] = pd.to_datetime(df['[Visits]Time Out'], format='%H:%M:%S')
except ValueError:
    problematic_rows = pd.to_datetime(df['[Visits]Time Out'], format='%H:%M:%S', errors='coerce').isna()
    df = df.loc[~problematic_rows]
    df['[Visits]Time Out'] = pd.to_datetime(df['[Visits]Time Out'], format='%H:%M:%S')
df['[Visits]Date In'] = pd.to_datetime(df['[Visits]Date In'], format='%m/%d/%Y')

# Reset index & display results
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19683 entries, 0 to 19682
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   [Students]Custom Field1  19616 non-null  object        
 1   [CenterReasons]Reason    19367 non-null  object        
 2   [Visits]Time In          19683 non-null  datetime64[ns]
 3   [Sections]Term ID        19683 non-null  object        
 4   [Students]Class          19616 non-null  object        
 5   [Visits]Time Out         19683 non-null  datetime64[ns]
 6   [Subcenters]Name         19683 non-null  object        
 7   [Visits]Date In          19683 non-null  datetime64[ns]
 8   [Students]College        19616 non-null  object        
 9   [Students]Major          19616 non-null  object        
 10  [Students]Ethnicity      19202 non-null  object        
 11  [Students]EMCCGPA        19683 non-null  object        
 12  [Students]Custom Field6  804 non

In [8]:
df['[Visits]Date In'].max()

Timestamp('2023-04-30 00:00:00')

In [4]:
# New feature: minutes spent (time out - time in)
df['mins_spent'] = round((df['[Visits]Time Out'] - df['[Visits]Time In']).dt.total_seconds() / 60, 2)
# New feature: year of visit
df['visit_year'] = df['[Visits]Date In'].dt.year
# Map class status (0: FR, 1: SO, 2: JR, 3: SR, 4: GR)
df['[Students]Class'] = df['[Students]Class'].map({'FR': 0, 'SO': 1, 'JR': 2, 'SR': 3, 'GR': 4})
# Map student athlete status (0: not student athlete, 1: student athlete)
df['[Students]Custom Field6'] = df['[Students]Custom Field6'].notnull().astype(int)
# Merge similar entries 
df['[Students]College'] = df['[Students]College'].map({'Liberal Arts&Sciences': 'Liberal Arts and Sciences', 
                                                       'Education': 'College of Education'}).fillna(df['[Students]College'])
df['[Students]Ethnicity'] = df['[Students]Ethnicity'].map({'African American': 'Black', 
                                                           'Hispanic/Latino': 'Hispanic', 
                                                           'Multiple': 'Mixed',
                                                           'African-AmericanAmerican IndianAsian': 'Mixed',
                                                           'American Indian/Native Alaskan': 'Native American',
                                                           'Native Hawaiian/Pacific Island': 'Native American',}).fillna(df['[Students]Ethnicity'])

# Drop, rename, reorder columns
df.drop(columns=['[Visits]Time In', '[Visits]Time Out', '[Visits]Date In', '[Sections]Term ID', '[Students]Custom Field1'], inplace=True)
df.rename(columns={'[Students]Major': 'major',
                   '[Faculty]FullName': 'professor',
                   '[CenterReasons]Reason': 'visit_purpose',
                   '[Students]College': 'college',
                   '[Students]Class': 'class',
                   '[Students]EMCCGPA': 'gpa',
                   '[Subcenters]Name': 'visit_location',
                   '[Students]Ethnicity': 'ethnicity',
                   '[Students]Custom Field6': 'student_athlete'}, inplace=True)
df = df[['visit_year', 'visit_location', 'visit_purpose', 'college', 'major', 'class', 'ethnicity', 'student_athlete', 'professor', 'gpa', 'mins_spent']]

# Reset index & display results
df = df.reset_index(drop=True)
df.head()

,visit_year,visit_location,visit_purpose,college,major,class,ethnicity,student_athlete,professor,gpa,mins_spent
0,2018,Reed Fourth,Writing,Liberal Arts and Sciences,English,2.0,White,1,"Ploetz, Elmer",2.12,93.60
1,2018,Reed Fourth,Writing,Liberal Arts and Sciences,English,2.0,White,1,"FitzPatrick, Cornelius",2.12,90.00
2,2018,Reed Fourth,Writing,Liberal Arts and Sciences,English,2.0,White,1,"FitzPatrick, Cornelius",2.12,176.75
3,2018,Reed Fourth,Writing,Liberal Arts and Sciences,English,2.0,White,1,"FitzPatrick, Cornelius",2.12,78.93
4,2018,Reed Fourth,Writing,Liberal Arts and Sciences,English,2.0,White,1,"FitzPatrick, Cornelius",2.12,90.00


In [5]:
# Replace NaN values in 'visit_purpose' based on 'student_athlete'
df['visit_purpose'] = df.apply(lambda row: 'Study Hall (Athlete)' if row['student_athlete'] == 1 and pd.isnull(row['visit_purpose'])
                               else 'General Tutoring' if row['student_athlete'] == 0 and pd.isnull(row['visit_purpose']) 
                               else row['visit_purpose'], axis=1)
# Drop rows with missing values
df.dropna(inplace=True)
# Convert dtype
df['class'] = df['class'].astype(int)
df['gpa'] = df['gpa'].astype(float)
# Remove rows with gpa: 0.00, or mins_spent <= 0
df = df[(df['gpa'] != 0) & (df['mins_spent'] > 0)]
# Reset index & display results
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18740 entries, 0 to 18739
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   visit_year       18740 non-null  int64  
 1   visit_location   18740 non-null  object 
 2   visit_purpose    18740 non-null  object 
 3   college          18740 non-null  object 
 4   major            18740 non-null  object 
 5   class            18740 non-null  int64  
 6   ethnicity        18740 non-null  object 
 7   student_athlete  18740 non-null  int64  
 8   professor        18740 non-null  object 
 9   gpa              18740 non-null  float64
 10  mins_spent       18740 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 1.6+ MB


In [ ]:
# Export cleaned dataframe as CSV
df.to_csv('data.csv')